Yunhu's product stock model defines several model parameters:
- Product demand in each year (globally)
- End-of-life product waste flow each year (globally)
- Composition of polymers within each product

The product demand is broken down down by region, polymer, sector and year. We want to sum the regions and polymers to get the demand by product for each year.

Separately, we want to find the composition of polymers for each product by year (although we are not changing it over time at the moment).

Finally, the end of life flow is further broken down by the time-period cohort which is leaving the stock. We don't need that for the calculator, so sum that out.

We also need to downsample to include data only for the years modelled in the calculator (2020 to 2050, every 5 years).

In [ ]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
data = loadmat("products_BAU.mat")

In [ ]:
data.keys()

In [ ]:
t = np.arange(123) + 1978

In [ ]:
t_idx = [list(t).index(y) for y in range(2020, 2051, 5)]
t_idx

## Inputs

Total input to each sector

In [ ]:
def flatten2cells(a):
    """Flatten 2 extra dimensions in mat file cell arrays."""
    return np.stack([
        np.stack([
            a[0, i][0, j]
            for j in range(a[0, i].shape[1])
        ])
        for i in range(a.shape[1])
    ])

def flatten3cells(a):
    """Flatten 3 extra dimensions in mat file cell arrays."""
    return np.stack([
        np.stack([
            np.stack([
                a[0, i][0, j][0, k]
                for k in range(a[0, i][0, j].shape[1])
            ])
            for j in range(a[0, i].shape[1])
        ])
        for i in range(a.shape[1])
    ])

In [ ]:
inp = flatten2cells(data["input2sector"])

In [ ]:
inp.shape

8 regions (m), 14 polymers (j), 8 sectors (i), 123 years (k) counting from 1978

In [ ]:
sectors = """\
Packaging
Transportation
Buildings and constructions
Electrical and electronic
Consumer & institutional
Industrial machinery
Textile
Others""".splitlines()

polymers = """\
LDPE
LLDPE
HDPE
PP
PS
PVC
PET
PUR
Polyester fiber
Polyamid fiber
Other fiber
Rubber (7)
Other thermoplastics (6)
Other thermosets (2)
""".splitlines()

regions = """\
North America
South America
Western and Central Europe
Central Europe and Eastern Asia
Africa
Middle East
Northeast Asia
South Asia and Pacific
""".splitlines()

In [ ]:
for i in range(len(sectors)):
    plt.plot(t, inp[0, :, i, :].sum(axis=0), label=sectors[i])
for i in t_idx:
    plt.axvline(t[i], c='k', lw=0.5)
plt.legend();

For each sector the virgin and recycled amounts should sum.

In [ ]:
inp_v = flatten2cells(data["input2sectorvirgin"])
inp_r = flatten2cells(data["input2sectorrecycled"])

In [ ]:
abs((inp_v + inp_r) - inp).max()

Yes -- they agree!

Sum the regions and polymers to get the total demand for each product/sector:

In [ ]:
demands = inp.sum(axis=0).sum(0)
plt.plot(t, demands[2])

In [ ]:
def convert_product_demand(filename, level):
    data = loadmat(filename)
    inp = flatten2cells(data["input2sector"])
    demands = inp.sum(axis=0).sum(axis=0)[:, t_idx]
    df = pd.DataFrame([
        [level, f"Z_product[{i}]"] + list(demands[i, :])
        for i in range(len(sectors))
    ], columns=["level", "param"] + [t[i] for i in t_idx])
    df["level"] = df["level"].astype(int)
    return df

In [ ]:
Z_product_data = pd.concat([
    convert_product_demand("products_BAU.mat", 1),
    #convert_product_demand("Constant Demand_stephen.mat", 2),
    convert_product_demand("products_demand_reduction_50_percent.mat", 2),
    convert_product_demand("products_demand_reduction_30_percent.mat", 3),
    convert_product_demand("products_demand_reduction_10_percent.mat", 4),
])

In [ ]:
df = Z_product_data.set_index(["param", "level"]).T
@interact(i=(0, 7))
def plot_region_sector_polymer_shares(i):
    df[f"Z_product[{i}]"].plot(title=sectors[i]);
    #plt.scatter(t_grid[0].ravel(), t_grid[1].ravel(), waste[m, j, i, :, :].T.ravel() / 3, alpha=0.3)
    #plt.ylabel("Cohort year");
    #plt.xlabel("Outflow year");

In [ ]:
Z_product_data

## Waste

In [ ]:
waste = flatten3cells(data["wastetemporary"])
waste.shape
# region, polymer, sector, t_out, t_cohort

In [ ]:
def convert_product_eol(filename, level):
    data = loadmat(filename)
    waste = flatten3cells(data["wastetemporary"])
    # Sum out region, polymer, and cohort
    waste = waste.sum(axis=0).sum(axis=0).sum(axis=2)[:, t_idx]
    df = pd.DataFrame([
        [level, f"Z_EOL[{i}]"] + list(waste[i, :])
        for i in range(len(sectors))
    ], columns=["level", "param"] + [t[i] for i in t_idx])
    df["level"] = df["level"].astype(int)
    return df

In [ ]:
Z_EOL_data = pd.concat([
    convert_product_eol("products_BAU.mat", 1),
    convert_product_eol("products_demand_reduction_50_percent.mat", 2),
    convert_product_eol("products_demand_reduction_30_percent.mat", 3),
    convert_product_eol("products_demand_reduction_10_percent.mat", 4),
])

In [ ]:
df = Z_EOL_data.set_index(["param", "level"]).T
@interact(i=(0, 7))
def plot_region_sector_polymer_shares(i):
    df[f"Z_EOL[{i}]"].plot(title=sectors[i]);

In [ ]:
Z_EOL_data

## Product composition

Generate process recipes for the combination of polymers into products.

We want this as a global average -- sum regions first.

In [ ]:
# sum over polymer types
inp_global = inp.sum(axis=0)
shares = inp_global / inp_global.sum(axis=0, keepdims=True)

In [ ]:
shares.shape

14 polymers (j), 8 sectors (i), 123 years (k) counting from 1978

In [ ]:
t[42]

In [ ]:
df = pd.DataFrame(shares[:, 3].T, index=t, columns=polymers)
df.plot.area()

Since constant composition is assumed from now into the future, use the latest values.

In [ ]:
shares_future = shares[:, :, -1]
shares_future.shape

Group together the polymer types where we want less detail.

In [ ]:
process_names = [
    "ProductionOfPackagingProducts",
    "ProductionOfTransportationProducts",
    "ProductionOfBuildingsAndConstructionProducts",
    "ProductionOfElectricalAndElectronicProducts",
    "ProductionOfConsumerAndInstitutionalProducts",
    "ProductionOfIndustrialMachinery",
    "ProductionOfTextileProducts",
    "ProductionOfOtherProducts",
]

object_name_map = {
    "LDPE": "ukf:LDPEPolyethylene",
    "LLDPE": "LLDPE",
    "HDPE": "ukf:HDPEPolyethylene",
    "PP": "ukf:PPPolypropylene",
    "PS": "ukf:PSPolystyrene",
    "PVC": "ukf:PVCPolyvinylChloride",
    "PET": "ukf:PETPolyethyleneTerephthalatePolyesters",
    "PUR": "ukf:Polyurethane",
    "Polyester fiber": "FibrePPA",
    "Polyamid fiber": "FibrePPA",
    "Other fiber": "FibrePPA",
    "Rubber (7)": "ukf:SyntheticRubbers",
    "Other thermoplastics (6)": "ukf:OtherPolymers",
    "Other thermosets (2)": "ukf:OtherPolymers",
}

In [ ]:
object_names = []
for v in object_name_map.values():
    if v not in object_names:
        object_names.append(v)
A = np.zeros((len(object_names), len(polymers)), dtype=int)
for k, v in object_name_map.items():
    A[object_names.index(v), polymers.index(k)] = 1
A

In [ ]:
shares_future_grouped = np.einsum("pi, qp -> qi", shares_future, A)

In [ ]:
shares_future[:, 0]

In [ ]:
shares_future_grouped[:, 0]

In [ ]:
print("""
<!-- WARNING: THESE DEFINITIONS ARE GENERATED AUTOMATICALLY -->
<!--          DO NOT EDIT BY HAND                           -->
<!--                                                        -->
<!-- Generated by "Convert product model.ipynb"             -->

""")
for k in range(len(process_names)):
    p = process_names[k]
    output_obj = p.replace("ProductionOf", "")
    print(f"```{{system:process}} {p}")
    print(f"---")
    print(f"produces: |")
    print(f"    {output_obj:<50} = 1 kg")
    print(f"consumes: |")
    for m in range(len(object_names)):
        input_obj = object_names[m]
        amount = shares_future_grouped[m, k]
        if amount > 1e-3:
            print(f"    {input_obj:<50} = {amount:.3f} kg")
    print(f"---")
    print(f"```")
    print()

print("""
<!-- WARNING: THESE DEFINITIONS ARE GENERATED AUTOMATICALLY -->
<!--          DO NOT EDIT BY HAND                           -->
<!--                                                        -->
<!-- Generated by "Convert product model.ipynb"             -->

""")
    
for k in range(len(process_names)):
    p = process_names[k]
    input_obj = p.replace("ProductionOf", "")
    print(f"```{{system:process}} EOLProcessing{input_obj}")
    print(f"---")
    print(f"consumes: |")
    print(f"    EOL{input_obj:<47} = 1 kg")
    print(f"produces: |")
    for m in range(len(object_names)):
        output_obj = object_names[m].replace("ukf:", "") + "AtEOL"
        amount = shares_future_grouped[m, k]
        if amount > 1e-3:
            print(f"    {output_obj:<50} = {amount:.3f} kg")
    print(f"---")
    print(f"```")
    print()